In [19]:
import xarray as xr
import ocf_blosc2
import pandas as pd

In [75]:
ds_ec_ext = xr.open_zarr("/mnt/disks/gcp_data_v3/nwp/ecmwf/UK_v3/ECMWF_2022.zarr")
ds_ec_old = xr.open_zarr("/mnt/disks/gcp_data_v3/nwp/ecmwf/UK_v2.zarr")

ds_mo_ext = xr.open_zarr("/mnt/disks/gcp_data_v3/nwp/ukv/UKV_v8/UKV_2022.zarr")
ds_mo_old = xr.open_zarr("/mnt/disks/gcp_data_v3/nwp/ukv/UKV_v7/UKV_2022.zarr")

In [ ]:
ds_mo_old

In [ ]:
ds_mo_ext.init_time.values

In [94]:
init_time = '2022-12-12T12:00:00.000000000' #ds_ec_ext.init_time.values[1]
lat = 51.60
lon = -0.70
x = 475872.699408
y = 195760.588558

In [ ]:
data_mo_old = ds_mo_old.sel(
    # init_time = init_time,
    x = x,
    y = y,
    method = 'nearest'
)

data_mo_old = data_mo_old.sel(
    variable = "sde"
)

data_mo_old.UKV.values

In [ ]:
# Get max value and index of "sde" variable across all dimensions
sde_max = data_mo_old.UKV.max().values
sde_max_idx = data_mo_old.UKV.argmax().values
print(f"Maximum value of 'sde' variable: {sde_max}")
print(f"Index of maximum value: {sde_max_idx}")


In [ ]:
# Now try the selection and plotting with the sorted dataset
data_ec_ext = ds_ec_ext.sel(
    init_time=init_time,
    latitude=lat,
    longitude=lon,
    method='nearest'
)
data_ec_ext = data_ec_ext.sel(
    variable="sd"
)



data_ec_old = ds_ec_old.sel(
    init_time=init_time,
    latitude=lat,
    longitude=lon,
    method='nearest'
)
data_ec_old = data_ec_old.sel(
    variable="sde"
)



data_mo_ext = ds_mo_ext.sel(
    init_time = init_time,
    x = lon,
    y = lat,
    method = 'nearest'
)

data_mo_ext = data_mo_ext.sel(
    variable = "sde"
)

data_mo_old = ds_mo_old.sel(
    init_time = init_time,
    x = lon,
    y = lat,
    method = 'nearest'
)

data_mo_old = data_mo_old.sel(
    variable = "sde"
)




# Create the plot
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[x.astype('timedelta64[h]').astype(int) for x in data_ec_ext.step.values],
    y=data_ec_ext.ECMWF_UK,
    name='ECMWF Extended'
))

# Add traces for both datasets
fig.add_trace(go.Scatter(
    x=[x.astype('timedelta64[h]').astype(int) for x in data_ec_old.step.values],
    y=data_ec_old.ECMWF_UK,
    name='ECMWF Old'
))

fig.add_trace(go.Scatter(
    x=[x.astype('timedelta64[h]').astype(int) for x in data_mo_ext.step.values],
    y=data_mo_ext.UKV,
    name='MO Extended'
))

# Add traces for both datasets
fig.add_trace(go.Scatter(
    x=[x.astype('timedelta64[h]').astype(int) for x in data_mo_old.step.values],
    y=data_mo_old.UKV,
    name='MO Old'
))



# Update layout
fig.update_layout(
    title=f'Snow Depth vs Step for {pd.Timestamp(init_time).strftime("%Y-%m-%d %H:%M")}<br>Location: {lat}°N, {lon}°E',
    xaxis_title='Forecast Step (hours)',
    yaxis_title='Snow Depth',
    showlegend=True,
    height=600,
    width=1200
)

fig.show()


In [ ]:
data_ec_old

In [ ]:
data_ec_ext